***1. Make Some Detections***

In [ ]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import time
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 
import matplotlib.pyplot as plt
import datetime
import time
%matplotlib 

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [ ]:
#LIVE
#cap = cv2.VideoCapture(0)
#happy/neutral/peaceful/suiron
filepath_input = "./emotions/happy.mp4"
filepath_output = "./result/happy.mp4"
cap = cv2.VideoCapture(filepath_input)
#video save
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 動画の画面横幅
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 動画の画面縦幅
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 総フレーム数
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) # フレームレート
size = (width, height)
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') # ファイル形式(ここではmp4)
writer = cv2.VideoWriter(filepath_output, fmt, frame_rate, size) # ライター作成

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                #mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,0,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        cv2.namedWindow("img", cv2.WINDOW_NORMAL)
        cv2.imshow('img', image)
        writer.write(image) 
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
writer.release()
cv2.destroyAllWindows()

In [ ]:
#results.face_landmarks.landmark[0].visibility
results.face_landmarks.landmark[0]

***2. Capture Landmarks & Export to CSV***

In [ ]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
landmarks

In [ ]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
#LIVE
#cap = cv2.VideoCapture(0)
#happy/neutral/peaceful/suiron
class_name = "peaceful"
filepath_input = "./emotions/peaceful.mp4"
#filepath_output = "./result/neutral.mp4"
cap = cv2.VideoCapture(filepath_input)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
            
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        #CSV出力のためのデータ作成
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
        
        cv2.namedWindow("img", cv2.WINDOW_NORMAL)
        cv2.imshow('img', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


***3. Train Custom Model Using Scikit Learn***

In [ ]:
#CSVだと最大列数制限を超えるのでtxtに変換→拡張子変更
df = pd.read_csv('coords.txt')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df[df['class']=='happy']

In [ ]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
X_train

In [ ]:
y_test

***3.2 Train Machine Learning Classification Model***

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models['rc'].predict(X_test)

***3.3 Evaluate and Serialize Model***

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:
y_test

In [ ]:
# save model
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

***4. Make Detections with Model***

In [ ]:
# load model
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
model

In [ ]:
prob_ = []
time_ = []
#LIVE
#cap = cv2.VideoCapture(0)
#happy/neutral/peaceful/suiron
start = time.time()
filepath_input = "./emotions/suiron.mp4"
filepath_output = "./result/suiron.mp4"
cap = cv2.VideoCapture(filepath_input)
#video save
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 動画の画面横幅
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 動画の画面縦幅
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 総フレーム数
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) # フレームレート
size = (width, height)
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') # ファイル形式(ここではmp4)
writer = cv2.VideoWriter(filepath_output, fmt, frame_rate, size) # ライター作成
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        #print(results)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=5, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=5, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=5, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=5, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=5, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=5, circle_radius=2)
                                 )
        # Export coordinates
        #try:
        # Extract Pose landmarks
        #print(results.pose_landmarks)
        #print(results.pose_landmarks.landmark)
        if results.pose_landmarks:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#       # Append class name 
#       row.insert(0, class_name)
            
#       # Export to CSV
#       with open('coords.csv', mode='a', newline='') as f:
#             csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#             csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            time_diff = time.time() - start
            time_.append(time_diff)
            prob_.append(body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            #print(coords)
            if body_language_class == 'happy':
                cv2.rectangle(image, (50,50), (860, 200), (80, 127, 255), -1)
            elif body_language_class == 'neutral':
                cv2.rectangle(image, (50,50), (860, 200), (211, 211, 211), -1)
            elif body_language_class == 'peaceful':
                cv2.rectangle(image, (50,50), (860, 200), (144, 238, 144), -1)
            
            #cv2.rectangle(image, (coords[0]+1000, coords[1]+1000), (coords[0]+len(body_language_class)*30+1500, coords[1]+150), (245, 117, 16),-1)
            
            
            cv2.putText(image, body_language_class, 
                        (350,150), cv2.FONT_HERSHEY_TRIPLEX, 3, (255, 255, 255), 3, cv2.LINE_AA)
            
            # Get status box
            
            # Display Class
            #cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 0), 5, cv2.LINE_AA)
            #cv2.putText(image, body_language_class.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 255, 255), 5, cv2.LINE_AA)
            
            # Display Probability
            #cv2.putText(image, 'PROB', (15,100), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 0), 5, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (90,150), cv2.FONT_HERSHEY_TRIPLEX, 3, (255, 255, 255), 3, cv2.LINE_AA)
            
            #except:
        else:
            pass
                        
        cv2.namedWindow("img", cv2.WINDOW_NORMAL)
        cv2.imshow('img', image)
        writer.write(image) 
        #time.sleep(0.2)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
writer.release()
cv2.destroyAllWindows()

In [ ]:
prob_ar = np.array(prob_)
print(len(prob_ar))
print(np.shape(prob_))
print(len(time_))

In [ ]:
df_prob = pd.DataFrame(prob_ar,columns=['happy', 'neutral', 'peaceful'])
df_prob

In [ ]:
time_ar = np.array(time_)
df_time = pd.DataFrame(time_ar,columns=['time'])
df_time

In [ ]:
df_sum = pd.concat([df_time, df_prob],axis=1)
df_sum_list = df_sum.values.tolist()
df_sum_list_ar = np.array(df_sum_list)
print(df_sum_list_ar[:,0])

In [ ]:
with open('plot_prob.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(df_sum_list)

In [ ]:
with open('plot_prob.txt') as f:
    reader = csv.reader(f)
    prob__ = [row for row in reader]
    #for row in reader:
        #prob__.append(row)

for i in range(len(prob__)):
    for j in range(len(prob__[i])):
        prob__[i][j] = float(prob__[i][j])

prob__ar = np.array(prob__)
#int_list = [int(i) for i in sample_list[0]]   
print(prob__ar[:,1])
print(np.shape(prob__ar))


In [ ]:
# step2 グラフフレームの作成
fig, ax = plt.subplots(figsize = (18,5),facecolor="black")
#fig, ax = plt.subplots(figsize = (18,5),facecolor="darkslategray")
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_color('white')
plt.gca().spines['left'].set_color('white')
plt.gca().xaxis.label.set_color('red')
ax.set_facecolor('black')
ax.legend(facecolor="black",edgecolor="black",labelcolor = 'white')

for i in range(len(time_)):
    # 値を更新
    time_diff_ = prob__ar[:,0]
    y1 = prob__ar[:,1]
    y2 = prob__ar[:,2]
    y3 = prob__ar[:,3]

    # グラフ描画
    #happy/victorius/sad/surprised/等々
    #happy
    happy_, = ax.plot(time_diff_, y1,label='happy', color='orange', linestyle='-',linewidth=1,alpha=0.8)
    #neutral
    neutral_, = ax.plot(time_diff_, y2,label='neutral', color='gray', linestyle='-',linewidth=1,alpha=0.8)
    #peaceful
    peaceful_, = ax.plot(time_diff_, y3,label='peaceful',color='lightgreen', linestyle='-',linewidth=1,alpha=0.8)
    
    ax.set_xlim(time_diff-10, time_diff+1)
    
    ax.set_ylim(0,20)
    ax.set_xlabel('Time [$sec$]',color='white')
    ax.set_ylabel('Proba',color='white')
    
    ax.tick_params(color='white')
    #ax.set_yticks(color='white')
    ax.legend(facecolor="darkslategray",edgecolor="darkslategray",labelcolor = 'white',bbox_to_anchor=(1.05, 1))
    #plt.grid()
    # 0.01秒停止
    plt.pause(0.05)
    happy_.remove()
    neutral_.remove()
    peaceful_.remove()